### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

In [1]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

/Users/won/dev/00_codeit/0_mission/200_DL_RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
import json

BASE_DIR = Path.cwd().parent  # /codeit-part3-team4
RAW_FOLDER = BASE_DIR / "data/raw/files"  # 수정된 경로

# 폴더에서 PDF 목록 가져오기
def get_pdf_paths(folder_path: Path | str) -> list[Path]:
    folder = Path(folder_path)
    pdf_paths = [p for p in folder.glob("*.pdf")]
    return sorted(pdf_paths)

PDF_PATH_LIST = get_pdf_paths(RAW_FOLDER)

In [4]:
# PDF to text
def extract_text(pdf_path: Path | str) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    return "\n".join(texts)

In [5]:
# Chunking
def chunk(text: str, size: int = 800) -> list[str]:
    return [text[i:i+size] for i in range(0, len(text), size)]

In [6]:
# 임베딩 및 인덱스 만들기
def build_index(chunks: list[str]) -> tuple:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embs = model.encode(chunks)

    index = faiss.IndexFlatL2(embs.shape[1])
    index.add(np.array(embs).astype("float32"))
    return index, model, chunks

In [7]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

다음 기준에 따라 **핵심 정보만 구조적으로 요약**해라.

[요약 규칙]
- 추측하지 말고, 문서에 명시된 내용만 사용
- 불필요한 설명, 배경, 미사여구 제거
- 사업 참여 판단에 필요한 정보 위주로 요약
- 항목이 없으면 "명시 없음"으로 표기

[출력 형식]
아래 각 항목은 **질문에 답하는 형태**로 작성하되,
반드시 `key: 답변` 형식으로 출력하라.
(예시) project_name: 고려대학교 차세대 포털·학사 정보시스템 구축 사업

project_name: 사업(용역)명은 무엇인가?
agency: 발주 기관(수요기관)은 어디인가?
purpose: 사업 목적(추진 배경)은 무엇인가?
budget: 총 사업 예산(사업비)은 얼마인가?
base_amount: 추정 가격/기초금액은 얼마인가?
contract_type: 계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?
deadline: 입찰/제안서 제출 마감일시는 언제인가?
submission_method: 제출 장소 또는 제출 방법(온라인/방문/우편)은 무엇인가?
submission_docs: 제출 서류 목록은 무엇인가?
format_requirement: 제안서(기술/가격) 제출 파일 형식이나 양식은 무엇인가?
duration: 사업 수행 기간은 얼마나 되는가?
start_date: 사업 착수(시작) 예정일은 언제인가?
end_date: 사업 종료(납품) 예정일은 언제인가?
scope: 제안 요청 범위(Scope/업무 범위)는 무엇인가?
requirements_must: 필수 요구사항(기능/성능/보안 등)은 무엇인가?
requirements_optional: 선택 요구사항(우대/가점)에는 무엇이 있는가?
eval_items: 평가 항목(기술/가격 등) 구성은 어떻게 되는가?
eval_weights: 평가 배점(점수) 비율은 어떻게 되는가?
price_eval: 가격 평가 방식(최저가/협상 등)은 무엇인가?
presentation: 제안 발표(PT/발표평가)가 있는가? 있다면 일정은?
qa_period: 질의응답(Q&A) 접수 기간은 언제인가?
qa_method: 질의응답 제출 방법/양식은 무엇인가?
briefing: 현장설명회가 있는가? 있다면 일정/장소는?
eligibility: 참가 자격 요건(면허/실적/인증/등급)은 무엇인가?
consortium: 공동수급/하도급/컨소시엄 가능 여부는?
staffing: 필수 인력 요건(PM/보안담당 등)이 있는가?
security: 보안/개인정보 요구사항(인증, ISMS 등)은 무엇인가?
deliverables: 납품물(산출물) 목록은 무엇인가?
maintenance: 유지보수/운영 조건(기간/범위)은 무엇인가?
contact: 문의처(담당자/전화/이메일)는 무엇인가?

[컨텍스트]
{context}

이제 위 형식에 맞춰 한국어로 요약해라.
"""

In [8]:
# 쿼리 답변
def answer(index, model, chunks, query: str, top_k: int = 15) -> str:
    q_emb = model.encode([query])
    _, I = index.search(np.array(q_emb).astype("float32"), top_k)
    
    context = "\n\n".join(chunks[i] for i in I[0])
    prompt = RFP_PROMPT.format(context=context)
    
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )
    return resp.choices[0].message.content

In [9]:
# 데이터 폴더 설정
docs = get_pdf_paths(RAW_FOLDER)
print(f"발견된 PDF: {len(docs)}개")
for i, doc in enumerate(docs):
    print(f"{i}: {doc.name}")

발견된 PDF: 100개
0: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
1: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
2: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
3: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
4: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
6: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
7: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
8: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
9: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
10: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
11: 경희대학교_[입찰공고] 산학협력단 정보시스

In [11]:
# 전체 문서별 인덱스 저장
doc_indexes = {}

for doc_path in docs:
    print(f"처리 중: {doc_path.name}")

    text = extract_text(doc_path)
    chunks = chunk(text)
    index, model, chunks = build_index(chunks)
    doc_indexes[doc_path] = (index, model, chunks)

    break
print("모든 문서 인덱싱 완료!")

처리 중: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1891.05it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


모든 문서 인덱싱 완료!


In [12]:
# 테스트 1: 단일 문서 (docs[0])
if docs:
    test_doc = docs[0]
    print(f"\n=== {test_doc.name} 분석 ===")
    index, model, chunks = doc_indexes[test_doc]
    result = answer(index, model, chunks, "RFP 분석해줘")
    print(result)


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: 복수의결권 및 스톡옵션 시스템 구축 고도화 용역사업  
agency: 중소벤처기업부  
purpose: 복수의결권 및 스톡옵션 신고 시스템 구축  
budget: 명시 없음  
base_amount: 명시 없음  
contract_type: 명시 없음  
deadline: 명시 없음  
submission_method: 명시 없음  
submission_docs: 제안서, 사업수행계획서, 요구사항 정의서, UI 정의서, 인터페이스 설계서 등  
format_requirement: 파일 형식 미제시  
duration: 명시 없음  
start_date: 명시 없음  
end_date: 명시 없음  
scope: 복수의결권 및 스톡옵션 관련 기능 구축  
requirements_must: OWASP 10개 조항 준수, 개인정보 처리단계별 기술적 보호조치, 전자정부 표준 프레임워크 적용  
requirements_optional: 명시 없음  
eval_items: 기술, 가격 평가  
eval_weights: 명시 없음  
price_eval: 명시 없음  
presentation: 명시 없음  
qa_period: 명시 없음  
qa_method: 명시 없음  
briefing: 명시 없음  
eligibility: 명시 없음  
consortium: 공동수급 가능  
staffing: 필수 인력 요건 명시 없음  
security: 보안 요구사항 준수  
deliverables: 각종 보고서 및 요구사항 문서  
maintenance: 명시 없음  
contact: 명시 없음  


In [ ]:
# # 테스트 2: 전체 문서 루프 (비용 주의! 주석 해제 후 사용)
# # 실행 완료시 rfp_summaries.json 파일이 생성됩니다.

# results = {}
# for doc_path in docs:
#     print(f"\n=== {doc_path.name} 분석 ===")
#     index, model, chunks = doc_indexes[doc_path]
#     result = answer(index, model, chunks, "RFP 분석해줘")
#     results[doc_path.name] = result
#     print(result)

# # JSON 저장
# import json
# with open(BASE_DIR / "rfp_summaries.json", "w", encoding="utf-8") as f:
#     json.dump(results, f, ensure_ascii=False, indent=2)
# print("전체 분석 완료!")


=== 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf 분석 ===
project_name: 고려대학교 차세대 포털·학사 정보시스템 구축 사업  
agency: 고려대학교  
purpose: 학령인구 감소 및 교육환경 변화에 대응하고, 분산된 시스템 통합 및 데이터 기반 대학경영 지원을 개선하기 위함  
budget: 11,270,000,000원 (V.A.T 포함, 3년 분할 지급)  
base_amount: 명시 없음  
contract_type: 제한 경쟁 입찰(협상에 의한 계약)  
deadline: 명시 없음  
submission_method: 명시 없음  
submission_docs: 명시 없음  
format_requirement: 명시 없음  
duration: 계약일로부터 24개월 이내  
start_date: 명시 없음  
end_date: 사업 종료일로부터 12개월의 무상유지보수 기간 포함  
scope: 포털 및 학사 정보시스템 관련 구축 및 통합  
requirements_must: 학적 기본관리, 학사일정관리 등 명시된 기능 요구 사항  
requirements_optional: 명시 없음  
eval_items: 기술 평가, 가격 평가  
eval_weights: 명시 없음  
price_eval: 협상에 의한 가격 평가  
presentation: 명시 없음  
qa_period: 명시 없음  
qa_method: 명시 없음  
briefing: 명시 없음  
eligibility: 명시 없음  
consortium: 명시 없음  
staffing: 명시 없음  
security: 명시 없음  
deliverables: 프로그램 목록, 프로그램명세서, 화면정의서, 사용자매뉴얼 등  
maintenance: 하자보수 계획 포함, 사업 종료 후 무상유지보수 12개월  
contact: 명시 없음  

=== 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf 분석 ===
project_name: 2025년